In [3]:
from collections import Counter
from string import digits
from pprint import pprint
import nltk
import requests

Buscar um conjunto de discursos de exemplo:

In [4]:
json = requests.get('http://babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2').json()
speeches = [data['content'] for data in json['results']]
pprint(speeches)

['Sr. Presidente, caros Deputados, população brasileira que nos acompanha '
 'pela   TV Câmara   , hoje é um dia muito importante para o nosso País, pois '
 'é um dia histórico.    Isso a que nós estamos assistindo é estarrecedor! '
 'Este já é o mais longo processo de cassação vivido nesta Casa. Foram muitas '
 'as manobras e muitas as tentativas de evitar a cassação do mandato do '
 'Deputado Eduardo Cunha.    Nós estamos aqui hoje para defender a cassação do '
 'mandato do Deputado Eduardo Cunha, porque ele fez muito mal ao Brasil. Além '
 'de todos os seus atos de corrupção e de tudo aquilo que foi denunciado, em '
 'nenhum momento o Deputado Eduardo Cunha apresentou algo que pudesse provar a '
 'sua inocência. Em vez disso, no Conselho de Ética e na CCJ o que o Deputado '
 'Eduardo Cunha fez foram ameaças a seus colegas, dizendo:   "Hoje sou eu. '
 'Amanhã serão vocês!"      De fato, Deputado Luiz Couto, o povo brasileiro e '
 'nós queremos saber quem são os outros Deputados que o

## Tokenize
Transforma o texto em uma lista de tokens.

In [5]:
def tokenize(text):
    return nltk.tokenize.word_tokenize(text, language='portuguese')

## Stemming
Processo de reduzir as palavras ao seu radical. Pode ser passado um dicionário de `Counters`, para poder manter a referência da palavra original.

In [6]:
def stemmize(token, stem_reference=None):
    token = token.casefold()
    stemmer = nltk.stem.RSLPStemmer()
    stemmed = stemmer.stem(token)

    if stem_reference is not None:
        reference = stem_reference.get(stemmed, Counter())
        reference.update([token])
        stem_reference[stemmed] = reference

    return stemmed

In [7]:
def stemmize_stopwords(stopwords=None):
    default_stopwords = nltk.corpus.stopwords.words('portuguese')
    if stopwords is not None:
        default_stopwords += stopwords

    return list(set([
        stemmize(stopword)
        for stopword in default_stopwords
    ]))


## Stopwords
Definir o conjunto de stopwords de acordo com a classe gramatical das palavras.

In [8]:
from nltk.corpus import floresta
twords = floresta.tagged_words()
twords[:50]

def simplify_tag(tag):
    if "+" in tag:
        return tag[tag.index("+")+1:]
    else:
        return tag

twords[:20]

[('Um', '>N+art'),
 ('revivalismo', 'H+n'),
 ('refrescante', 'N<+adj'),
 ('O', '>N+art'),
 ('7_e_Meio', 'H+prop'),
 ('é', 'P+v-fin'),
 ('um', '>N+art'),
 ('ex-libris', 'H+n'),
 ('de', 'H+prp'),
 ('a', '>N+art'),
 ('noite', 'H+n'),
 ('algarvia', 'N<+adj'),
 ('.', '.'),
 ('É', 'P+v-fin'),
 ('uma', 'H+num'),
 ('de', 'H+prp'),
 ('as', '>N+art'),
 ('mais', '>A+adv'),
 ('antigas', 'H+adj'),
 ('discotecas', 'H+n')]

In [9]:
stopwords = []

valid_tags = ['adj', 'n', 'prop', 'nprop', 'est', 'npro', 'v-fin', 'v-inf', 'v-ger', 'v-pcp', 'vaux', 'v', 'vp', 'pcp']

for (word, tag) in twords:
    tag = simplify_tag(tag)
    word = word.casefold().replace('_', ' ')
    if tag not in valid_tags:
        stopwords.append(word)
stopwords = list(set(stopwords))

## Bag of Words (BOW)
Retorna um `Counter` com os `stems` de todos os termos que não estão listados na lista de stopwords. Também pode ser passado um parâmetro `method` para definir a forma que a frequência será computada (somente o método `frequency` foi implementado por enquanto). Além disso, também retorna um dicionário de `Counter`, como referencia dos termos não "stemmizados".

In [17]:
EXTRA_STOPWORDS = stopwords + [',', '.', 'srs', 'sr.', 'sra.', 'deputado', 'presidente', 'é', ':', "''", '`', '!', '``', '?', 'nº', 's.a.', 'quero', 'grande', 'dia', 'disse', 'pode', 'nesta', 'vamos', 'vai', 'vez', 'sras', 'dizer', 'falar', 'dar', 'chegou', 'mostrar', 'desses', 'coloca', 'deixou', '%', 'coisa', 'acharam', 'ficar', 'v.exa.', 'conclusao']

def bow(text, method='frequency'):
    tokens = tokenize(text)
    stopwords = stemmize_stopwords(
        stopwords=EXTRA_STOPWORDS
    )
    stem_reference = {}

    text_bow = Counter([
        stemmize(token) for token in tokens
        if stemmize(token, stem_reference=stem_reference) not in stopwords
    ])
    return text_bow, stem_reference

## Most Common Words
Retorna uma lista de tuplas com os termos mais frequentes e suas frequencias. Pode receber como parâmtro a quantidade de termos.

In [18]:
def most_common_words(text_bow, reference, n=None):
    most_common = []

    for token in text_bow.most_common(n):
        stem, frequency = token

        # reference[stem] is a Counter and most_comon(1) return a list
        # of tuples: ('word', occurrences)
        word = reference[stem].most_common(1)[0][0]
        most_common.append((word, frequency))
    return most_common

In [19]:
def remove_numeric_characters(text):
    remove_digits = str.maketrans('', '', digits)
    return text.translate(remove_digits)
text = remove_numeric_characters(' '.join(speeches))
pprint(text)

('Sr. Presidente, caros Deputados, população brasileira que nos acompanha '
 'pela   TV Câmara   , hoje é um dia muito importante para o nosso País, pois '
 'é um dia histórico.    Isso a que nós estamos assistindo é estarrecedor! '
 'Este já é o mais longo processo de cassação vivido nesta Casa. Foram muitas '
 'as manobras e muitas as tentativas de evitar a cassação do mandato do '
 'Deputado Eduardo Cunha.    Nós estamos aqui hoje para defender a cassação do '
 'mandato do Deputado Eduardo Cunha, porque ele fez muito mal ao Brasil. Além '
 'de todos os seus atos de corrupção e de tudo aquilo que foi denunciado, em '
 'nenhum momento o Deputado Eduardo Cunha apresentou algo que pudesse provar a '
 'sua inocência. Em vez disso, no Conselho de Ética e na CCJ o que o Deputado '
 'Eduardo Cunha fez foram ameaças a seus colegas, dizendo:   "Hoje sou eu. '
 'Amanhã serão vocês!"      De fato, Deputado Luiz Couto, o povo brasileiro e '
 'nós queremos saber quem são os outros Deputados que o

In [20]:
text_bow, reference = bow(' '.join(speeches))
common = most_common_words(text_bow, reference)
pprint(common)

[('brasil', 70),
 ('cunha', 42),
 ('eduardo', 38),
 ('votar', 34),
 ('governo', 32),
 ('golpe', 26),
 ('cassação', 25),
 ('trabalho', 23),
 ('país', 22),
 ('fazer', 21),
 ('pública', 20),
 ('servidores', 20),
 ('importante', 19),
 ('política', 18),
 ('federal', 18),
 ('parlamentares', 17),
 ('ministro', 16),
 ('justiça', 16),
 ('povo', 15),
 ('ministério', 15),
 ('empresa', 15),
 ('cidadão', 15),
 ('petrucio', 15),
 ('mandato', 14),
 ('santas', 14),
 ('conselho', 13),
 ('nacional', 13),
 ('passada', 13),
 ('processo', 12),
 ('momento', 12),
 ('impeachment', 12),
 ('hora', 12),
 ('questão', 12),
 ('pauta', 12),
 ('cargos', 12),
 ('atletas', 12),
 ('defender', 11),
 ('dilma', 11),
 ('saúde', 11),
 ('recursos', 11),
 ('tempo', 11),
 ('fazenda', 11),
 ('posse', 11),
 ('tentativa', 10),
 ('rio', 10),
 ('anos', 10),
 ('parlamento', 10),
 ('luta', 10),
 ('lei', 10),
 ('população', 9),
 ('manobras', 9),
 ('ética', 9),
 ('sul', 9),
 ('sociedade', 9),
 ('participei', 9),
 ('verdade', 9),
 ('pres

In [21]:
# Normalize common words
max_value = max(text_bow.values())
for token in text_bow:
    text_bow[token] /= max_value
text_bow.most_common(20)

[('brasil', 1.0),
 ('cunh', 0.6),
 ('eduard', 0.5428571428571428),
 ('vot', 0.4857142857142857),
 ('govern', 0.45714285714285713),
 ('golp', 0.37142857142857144),
 ('cass', 0.35714285714285715),
 ('trabalh', 0.32857142857142857),
 ('país', 0.3142857142857143),
 ('faz', 0.3),
 ('públic', 0.2857142857142857),
 ('serv', 0.2857142857142857),
 ('import', 0.2714285714285714),
 ('polít', 0.2571428571428571),
 ('feder', 0.2571428571428571),
 ('parlament', 0.24285714285714285),
 ('ministr', 0.22857142857142856),
 ('justiç', 0.22857142857142856),
 ('pov', 0.21428571428571427),
 ('minist', 0.21428571428571427)]

In [22]:
common = most_common_words(text_bow, reference)
pprint(common[:20])

[('brasil', 1.0),
 ('cunha', 0.6),
 ('eduardo', 0.5428571428571428),
 ('votar', 0.4857142857142857),
 ('governo', 0.45714285714285713),
 ('golpe', 0.37142857142857144),
 ('cassação', 0.35714285714285715),
 ('trabalho', 0.32857142857142857),
 ('país', 0.3142857142857143),
 ('fazer', 0.3),
 ('pública', 0.2857142857142857),
 ('servidores', 0.2857142857142857),
 ('importante', 0.2714285714285714),
 ('política', 0.2571428571428571),
 ('federal', 0.2571428571428571),
 ('parlamentares', 0.24285714285714285),
 ('ministro', 0.22857142857142856),
 ('justiça', 0.22857142857142856),
 ('povo', 0.21428571428571427),
 ('ministério', 0.21428571428571427)]
